Este notebook realiza a limpeza e o pré-processamento dos dados para cada dataset listado no arquivo de configuração. Ele remove colunas desnecessárias, lida com valores ausentes, e salva o DataFrame limpo em um formato eficiente na pasta de dados processados.

### Configuração e Importação de Bibliotecas

In [1]:
%run ../src/settings.py

In [2]:
import pandas as pd
from src.settings import load_config
from src.data_processing import load_raw_data, prepare_data, \
  save_processed_data

In [3]:
config = load_config()

# Define os parâmetros de processamento a partir do config
raw_data_path = config['data']['raw_path']
processed_data_path = config['data']['processed_path']
cols_to_drop = ["RendaMedia"]
id_column = "Cod_Setor"

In [ ]:
processing_results = []

for file_name in config['datasets']:
  try:
    # Carrega os dados brutos
    df_raw = load_raw_data(filename=file_name, data_path=raw_data_path)

    # Prepara os dados (limpeza e seleção de features)
    df_processed = prepare_data(df_raw, cols_to_drop=cols_to_drop)

    # Define o nome do arquivo de saída
    output_file_name = f"{file_name.split('.')[0]}.parquet"

    # Salva o resultado
    res = save_processed_data(df_processed, filename=output_file_name,
                              data_path=processed_data_path)

    processing_results.append(
      {'file_name': file_name,
       'file_processed': res,
       'total_rows': len(df_processed),
       'status': 'success',
       'error': None
       }
    )


  except (FileNotFoundError, ValueError) as e:
    processing_results.append(
      {'file_name': file_name,
       'file_processed': None,
       'total_rows': None,
       'status': 'error',
       'error': e}
    )
    continue

pd.DataFrame(processing_results)